### Load necessary libraries

In [1]:
import numpy as np
import pandas as pd
import neuralsens.partial_derivatives as ns
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn import set_config
set_config(display='diagram')
%matplotlib qt

### Create synthetic dataset to check behavior of functions

In [2]:
samples = 100000
n_columns = 8
sm = np.random.normal(size=(samples,n_columns))
df = pd.DataFrame(sm, columns=['X' + str(x) for x in range(1,n_columns+1)])

### Check behavior of Hessian function

#### Create output Y as linear function of inputs with some non-linear relationship

In [3]:
df['Y'] = - 0.4 * df.X1 ** 3 - 0.5 * df.X2 ** 2 + 0.7 * df.X3 * df.X4 + 0.1 * np.random.normal(size=(samples,)) 

#### Train MLP model using the data.frame created

In [4]:
## Create random 80/20 % split
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns != 'Y'].to_numpy(), df['Y'], test_size = 0.2, random_state = 5)

In [5]:
### Create MLP model
model = MLPRegressor(solver='sgd', # Update function
                    hidden_layer_sizes=[10], # #neurons in hidden layers
                    learning_rate_init=0.001, # initial learning rate
                    activation='logistic', # Logistic sigmoid activation function
                    alpha=0.01, # L2 regularization term
                    learning_rate='adaptive', # Type of learning rate used in training
                    max_iter=500, # Maximum number of iterations
                    batch_size=10, # Size of batch when training
                    tol=1e-2, # Tolerance for the optimization
                    validation_fraction=0.0, # Percentage of samples used for validation
                    n_iter_no_change=10, # Maximum number of epochs to not meet tol improvement
                    random_state=150)

# Train model
model.fit(X_train, y_train)

MLPRegressor(activation='logistic', alpha=0.01, batch_size=10,
             hidden_layer_sizes=[10], learning_rate='adaptive', max_iter=500,
             random_state=150, solver='sgd', tol=0.01, validation_fraction=0.0)

In [6]:
# Predict values to check model performance
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Obtain performance metrics
print("Training set: R2", r2_score(y_train, y_train_pred), "MSE", mean_squared_error(y_train, y_train_pred))
print("Test set: R2", r2_score(y_test, y_test_pred), "MSE", mean_squared_error(y_test, y_test_pred))

Training set: R2 0.9545304608269689 MSE 0.1566474580592462
Test set: R2 0.9489131534495605 MSE 0.1698521452372367


#### Execute hessian function and check sensitivity metrics

In [7]:
# Obtain parameters to perform hessian
wts = model.coefs_
bias = model.intercepts_
actfunc = ['identity',model.get_params()['activation'],model.out_activation_]
X = pd.DataFrame(X_train, columns=df.columns[df.columns != 'Y'])
y = pd.DataFrame(y_train, columns=['Y'])
sens_end_layer = 'last'
sens_end_input = False
sens_origin_layer = 0
sens_origin_input = True

In [8]:
hessian = ns.hessian_mlp(wts, bias, actfunc, X, y)

In [9]:
# Check sensitivity metrics
# For X1, X5 mean and std should be around 0
# For X2, mean should be around -1
# For X3, X4, mean should be around 0.7
hessian.summary()

Sensitivity analysis of [8, 10, 1] MLP network.

Sensitivity measures of each output:

$Y 

metric      mean                                                              \
input         X1        X2        X3        X4        X5        X6        X7   
X1      0.018614 -0.001185 -0.002118 -0.000475 -0.001352 -0.002201  0.002742   
X2     -0.001185 -0.930347 -0.003779  0.001203 -0.001259  0.000418 -0.000706   
X3     -0.002118 -0.003779  0.000500  0.622927 -0.001408 -0.002254  0.001124   
X4     -0.000475  0.001203  0.622927  0.001884 -0.000696  0.000677 -0.000856   
X5     -0.001352 -0.001259 -0.001408 -0.000696 -0.000348 -0.000192  0.000011   
X6     -0.002201  0.000418 -0.002254  0.000677 -0.000192 -0.000102  0.000022   
X7      0.002742 -0.000706  0.001124 -0.000856  0.000011  0.000022  0.000017   
X8     -0.001394  0.000232 -0.000238  0.000111  0.000111  0.000108 -0.000035   

metric                 std            ...                     mean_squared  \
input         X8        X1   

In [10]:
hessian.info()

Sensitivity analysis of [8, 10, 1] MLP network.

80000 samples

Sensitivities of each output (only 5 first samples):

$Y 

input        X1                                                              \
input        X1        X2        X3        X4        X5        X6        X7   
0      1.420800  0.039983 -0.117257  0.096633 -0.000918 -0.004460  0.003808   
1     -4.907888 -0.108965  0.016810 -0.034772  0.000626 -0.001111  0.003403   
2     -0.122712 -0.067455 -0.028222  0.082976 -0.005913 -0.004727  0.001736   
3      0.917230 -0.071788  0.145817 -0.140794 -0.003081 -0.004947  0.002342   
4      0.839122 -0.026569 -0.020253 -0.009926 -0.002104 -0.004192  0.002447   

input                  X2            ...        X7                  X8  \
input        X8        X1        X2  ...        X7        X8        X1   
0     -0.001607  0.039983 -0.908404  ...  0.000012 -0.000031 -0.001607   
1     -0.008674 -0.108965 -1.372393  ... -0.000001 -0.000046 -0.008674   
2      0.000823 -0.067455 -